# Scraping the schools and staff info of FCPS





In [ ]:
!pip install requests --upgrade -q

In [ ]:
import requests

In [ ]:
schools_url = 'https://www.fcps.edu/schools-centers'

In [ ]:
response = requests.get(schools_url)
response.status_code

In [ ]:
len(response.text)

In [ ]:
page_contents = response.text

In [ ]:
!pip install beautifulsoup4 --upgrade --quiet

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
doc = BeautifulSoup(page_contents,'html.parser')

In [ ]:
type(doc)

In [ ]:
school_selection = 'node__title'
repo_tags = doc.find_all('div', {'class':school_selection})

In [ ]:
repo_tags[0].text.strip()

In [ ]:
a_tags = repo_tags[0].find_all('a')

In [ ]:
a_tags[0].text.strip()

In [ ]:
a_tags[1]['href']

In [ ]:
repo_tags[0].find_all('a')[0].text.strip()

In [ ]:
schools_names = []
for tag in repo_tags:
    #print(a_tags.text.strip())
    schools_names.append(tag.text.strip())
    #print(tag)
print(schools_names)
len(schools_names)

In [ ]:
school_links=[]
base_url = 'https://www.fcps.edu'
for i in repo_tags:
    p= i.find('a')
    school_links.append(base_url + p['href'])
school_links

## getting the information of the staff

In [ ]:
staff_info_url = school_links[1]
staff_info_url

In [ ]:
response = requests.get(staff_info_url)

In [ ]:
response.status_code

In [ ]:
len(response.text)

In [ ]:
staff_doc = BeautifulSoup(response.text,'html.parser')


In [ ]:
staff_info = staff_doc.find_all('a',{'class': "button"})

In [ ]:
for i in range(len(staff_info)):
    if staff_info[i].text == 'Staff Directory':
        s_directory =staff_info[i]['href']

In [ ]:
s_directory

In [ ]:
response = requests.get(s_directory)

In [ ]:
response.status_code

In [ ]:
p = st

In [ ]:
response = requests.get(s_directory)
staff_list = BeautifulSoup(response.text,'html.parser')
next_page = staff_list.find_all('a',class_='pager__link pager__link--next')
next_page[0]['href']
next_page_link=s_directory+next_page[0]['href']
next_page_link

In [ ]:
staff_name = staff_list.find_all('td',class_="views-field views-field-field-last-name")

In [ ]:
staff_list = BeautifulSoup(response.text,'html.parser')
staff_name = staff_list.find_all('td',class_="views-field views-field-field-last-name")
staff_name_list=[]
for i in staff_name:
    staff_name_list.append(i.text)

In [ ]:
staff_name_list

In [ ]:
position_name = staff_list.find_all('td',class_="views-field views-field-field-staff-title")

In [ ]:
position_list=[]
for i in position_name:
    position_list.append(i.text)

In [ ]:
position_list

In [ ]:
contact_df = staff_list.find_all('td',class_="views-field views-field-rendered-item")

In [ ]:
contact_list = []
base_url = 'https://aldrines.fcps.edu'
for i in contact_df:
    contact_list.append(base_url + i.find('a')['href'])

In [ ]:
page_num = input("Enter the number of pages")
for i in range(int(page_num)):
    url='https://www.fcps.edu/schools-centers?items_per_page=10&keywords=&page='+str(i)
    print(url)
    


# Final code
## Putting it all together

- We have a funciton to get the list of topics
- We have a function to create a CSV file for scraped repos from a topics page
- Let's create a function to put them together

### In the  below function we are taking the list of schools under FCPS

In [ ]:
def get_schools_list(num_pages):
    schools_names = []
    school_links = []
    for i in range(int(num_pages)):
        url='https://www.fcps.edu/schools-centers?items_per_page=10&keywords=&page='+str(i)
        print(url)
        req = requests.get(url)
        content=BeautifulSoup(req.text,'html.parser')
        school_selection = 'node__title'
        repo_tags = content.find_all('div', {'class':school_selection})
        for tag in repo_tags:
        #print(a_tags.text.strip())
            schools_names.append(tag.text.strip())
        base_url = 'https://www.fcps.edu'
        for i in repo_tags:
            p= i.find('a')
            school_links.append(base_url + p['href'])  

    return schools_names,school_links



In [ ]:
school_names,school_links = get_schools_list(27)

### Here we are extracting the info of all the employee under FCPS

In [ ]:
def get_staff_info(school_links):
    staff_name_list=[]
    position_list=[]
    contact_list = []
    for i in school_links[1:]:
        
        print(i)
        
        staff_info_url = i
        staff_info_url
        response = requests.get(staff_info_url)
        staff_doc = BeautifulSoup(response.text,'html.parser')
        staff_info = staff_doc.find_all('a',{'class': "button"})
        

        for i in range(len(staff_info)):
            if staff_info[i].text == 'Staff Directory':
                s_directory =staff_info[i]['href']
                
        response = requests.get(s_directory)
        staff_list = BeautifulSoup(response.text,'html.parser')
        
        
        #staff_list = BeautifulSoup(response.text,'html.parser')
        staff_name = staff_list.find_all('td',class_="views-field views-field-field-last-name")
        
        for i in staff_name:
            staff_name_list.append(i.text)
        
        position_name = staff_list.find_all('td',class_="views-field views-field-field-staff-title")
        
        for i in position_name:
            position_list.append(i.text)
        contact_df = staff_list.find_all('td',class_="views-field views-field-rendered-item")
        
        base_url = 'https://aldrines.fcps.edu'
        for i in contact_df:
            contact_list.append(base_url + i.find('a')['href'])
            
        next_page = staff_list.find_all('a',class_='pager__link pager__link--next')
        #next_page_link = next_page[0]['href']
        next_page_link=s_directory+next_page[0]['href']
        print(next_page_link)
            
        while next_page_link:
            response = requests.get(next_page_link)
            staff_list = BeautifulSoup(response.text,'html.parser')
            #print('234567')

            #staff_list = BeautifulSoup(response.text,'html.parser')
            staff_name = staff_list.find_all('td',class_="views-field views-field-field-last-name")

            for i in staff_name:
                staff_name_list.append(i.text)
            print(staff_name_list)
            position_name = staff_list.find_all('td',class_="views-field views-field-field-staff-title")

            for i in position_name:
                position_list.append(i.text)
            contact_df = staff_list.find_all('td',class_="views-field views-field-rendered-item")

            base_url = 'https://aldrines.fcps.edu'
            for i in contact_df:
                contact_list.append(base_url + i.find('a')['href'])

            next_page = staff_list.find_all('a',class_='pager__link pager__link--next')
            #next_page_link = next_page[0]['href']
            if next_page != []:
                next_page_link=s_directory+next_page[0]['href']
            else:
                break
            
          
            
    return staff_name_list,position_list,contact_list


### now making the dict for further using into the python

In [ ]:
m,n,q = get_staff_info(school_links)

In [ ]:

dict = {
    
    'Employee name': m,
    'Position': n,
    'contace':q,
}

## Now we have to import the file into the system by using os and panda library

In [2]:
import os
import pandas as pd

In [5]:
df = pd.DataFrame(dict) 

df.to_csv('1.csv')